# &AElig;co
***Calc module***
<br/><br/>

✈️♻️

This code is part of the Master's Thesis "A Life Cycle Approach for Aircraft Ecodesign", by Giácomo Parolin.

``Please enter the following information:``

In [1]:
import os
os.chdir("C:/Users/giparoli/Documents/Projetos/AEco")

aircraft_type = 'pax'  #pax or cargo

input_path = './Data/A320_inputs.xlsx'
input_sheet = 'A320'
output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_A320.xlsx'

iterations = 1000
chunks = 1000  #'auto' or more than 500

In [2]:
from Tools import *
from Model import *

# LCI

In [3]:
inputs = read_inputs(input_path, input_sheet) #reads inputs
p = ParameterSet(inputs, iterations, chunks) #creates ParameterSet object
p = func_unit(aircraft_type, p) #creates funcional unit parameters (see Tools.py)

In [4]:
UP_dataframe = read_unit_processes(database_path) #reads Unit Processes from database
UP = unit_process_dataset(UP_dataframe) #creates Unit Processes xarray dataset

In [5]:
inventory = LCI(name=input_sheet, type=aircraft_type, iterations=iterations, UP=UP, parameters=p) #creates empty LCI object
inventory.run(); #runs LCI model (see Model.py)

# LCIA

In [9]:
MP_data, EP_data = read_CF(database_path) #reads MP and EP factors from database
CF = CharactFactors(MP_data, EP_data, UP.Substances) #creates CharactFactors object]
CF.MP.dataset['NLT'].data = np.negative(CF.MP.dataset['NLT'].data) #adjusting the NLT category CFs.
CF.MP.to_array();

In [10]:
aeco = LCIA.build(inventory, CF) #creating LCIA object

In [11]:
aeco.build_CTV(parameterset=p); #Calculate CTV results for all inputs with midpoint and endpoint results

# Export Results

In [53]:
aeco.save(output_path, LCI=False) #Export MP, EP, LCI and CTV result to NetCDF file.

In [22]:
#aeco.to_excel(output_path)